In [ ]:
# load example data from github
!git clone https://github.com/match-Aero/bibazu_example_data.git

In [ ]:
# downgrade tensorflow for compatibility with local machine
%pip install tensorflow==2.10.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

print("keras:",keras.__version__)
print("tf:",tf.__version__)

In [ ]:
import datetime
%load_ext tensorboard
log_dir = os.path.join(os.getcwd(),"logs","fit",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
#function to plot 9 samples from dataset
def plot_samples(dataset,title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title)
    for images, labels in dataset.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(int(np.argmax(labels[i])))
            plt.axis("off")

In [ ]:
#standard paths
synthetic_ds_path = "bibazu_example_data/dataset/Rk2i_synthetic/"
real_ds_path = "bibazu_example_data/dataset/Rk2i_real/"

In [ ]:
#load images into batched dataset objects

#dataset parameters
img_height = 224
img_width = 224
validation_split = 0.2

train_batch_size = 20
val_batch_size = 5
test_batch_size = 1

#synthetic dataset for training and validation
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory=synthetic_ds_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=train_batch_size,
    validation_split = validation_split,
    subset = 'training',
    seed = 10,
    color_mode = 'rgb',
    #shuffle = False,
    image_size=(img_height, img_width))

val_ds = keras.preprocessing.image_dataset_from_directory(
    directory=synthetic_ds_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=val_batch_size,
    validation_split = validation_split,
    subset = 'validation',
    seed = 10,
    color_mode = 'rgb',
    #shuffle = False,
    image_size=(img_height, img_width))

#real dataset for testing trained model
test_ds = keras.preprocessing.image_dataset_from_directory(
    directory=real_ds_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=test_batch_size,
    color_mode = 'rgb',
    shuffle = False,
    image_size=(img_height, img_width))

#plot_samples(train_ds,"pre-augmentation")

In [ ]:
#get number of classes for output layer size
dataset_labels = train_ds.class_names
num_classes = len(dataset_labels)

In [ ]:
train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
#augmentations
#augmentations can also be added as layer to the model, it is then performed by the GPU

#define augmentation set
data_augmentation = keras.Sequential([
        layers.RandomRotation(0.025),
        layers.RandomBrightness(0.4),
        layers.RandomContrast(0.2),
        layers.RandomTranslation(0.1,0.1),
        layers.RandomZoom(0.1)
        ])

#apply augmentation set to training dataset
train_ds_augmentend = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE)

#plot_samples(train_ds_augmentend,"augmented")

In [ ]:
normalization_layer = layers.Rescaling(1./255)
train_ds_augmentend = train_ds_augmentend.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

#plot_samples(train_ds_augmentend,"augmented")

In [ ]:
#load an empty MobileNet model
model = keras.applications.mobilenet.MobileNet()

#remove last 4 layers (determined in fine tuning)
x = model.layers[-5].output

#set proper size for last layer
x = tf.keras.layers.Reshape(target_shape=(1024,))(x)

#add dense-layer as output
output = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)

#set input and output layers
model = keras.models.Model(inputs=model.input, outputs=output)

#compile model, set learning parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%tensorboard --logdir logs

In [ ]:
#training
history = model.fit(x=train_ds_augmentend,
            steps_per_epoch=len(train_ds),
            validation_data=val_ds,
            validation_steps=len(val_ds),
            epochs=200,
            callbacks=[tensorboard_callback],
            verbose=2
           )
            #tf: 6m 20s  callbacks=[tensorboard_callback]
            #ohnetf: 3m45s


In [ ]:
#plot training history
plt.style.use("ggplot")
fig,axs = plt.subplots(1,2)

fig.set_figheight(2.5)
fig.set_figwidth(10)

axs[0].plot(history.history['accuracy'])
axs[0].plot(history.history['val_accuracy'])
axs[0].set_title('model accuracy')
axs[0].set_ylabel('accuracy')
axs[0].set_xlabel('epoch')
axs[0].legend(['train', 'val'], loc='lower right')

axs[1].plot(history.history['loss'])
axs[1].plot(history.history['val_loss'])
axs[1].set_title('model loss')
axs[1].set_ylabel('loss')
axs[1].set_xlabel('epoch')
axs[1].legend(['train', 'val'], loc='upper right')

plt.show()

In [ ]:
#save model
model_save_path = "trained_models"

if not model_save_path == "":
    keras.models.save_model(model,model_save_path)
    print("model saved to", model_save_path)

In [ ]:
#load previously trained model
model_load_path = model_save_path

if not model_load_path == "":
    model = keras.models.load_model(model_load_path,compile=False)
    model.compile()

In [ ]:
#do prediction on real dataset
prediction_ds = test_ds #dataset to perform prediction on

test_predictions = model.predict(x=prediction_ds, steps=len(prediction_ds), verbose=2)
predicted_labels = test_predictions.argmax(axis=1) #get labes for predictions

#get true labels from real dataset
ds_true_labes = tf.concat([y for x, y in prediction_ds], axis=0)
ds_true_labes = np.argmax(ds_true_labes, axis=1)

In [ ]:
#calculate confusion matrix
cm = confusion_matrix(y_true=ds_true_labes, y_pred=predicted_labels)

#plot confusion matrix
cm_plot = ConfusionMatrixDisplay(confusion_matrix = cm,display_labels = dataset_labels)
cm_plot.plot(cmap= plt.cm.Blues)
plt.grid(None)